In [46]:
import cv2
import numpy as np

# 读取原始图像
image = cv2.imread('D1.jpg')

In [47]:
# 将图像转换为灰度图像
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 二值化处理
ret, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

# 轮廓检测
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# 找到凹多边形的轮廓
outer_contour = max(contours, key=cv2.contourArea)

# 根据凹多边形的轮廓创建掩码
mask = np.zeros_like(gray)
cv2.drawContours(mask, [outer_contour], -1, (255), thickness=cv2.FILLED)

# 根据掩码裁剪图像
# cropped_image = cv2.bitwise_and(image, image, mask=mask)

# # 显示裁剪后的图像
# cv2.imshow('Cropped Image', cropped_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite('D1_cropped.jpg', mask)


True

In [52]:

h, w = image.shape[:2]
fov = np.pi / 2
focal_length = (w + h) / (2 * np.tan(fov / 2))

# 生成球形投影的坐标
map_x = np.zeros((h, w), dtype=np.float32)
map_y = np.zeros((h, w), dtype=np.float32)
for y in range(h):
    for x in range(w):
        if mask[y, x] == 255:  # 判断当前像素是否在指定的区域内
            theta = 2.0 * np.pi * (x / w - 0.5)
            phi = np.pi * (y / h - 0.5)
            x_sphere = np.sin(theta) * np.cos(phi)
            y_sphere = np.sin(phi)
            z_sphere = np.cos(theta) * np.cos(phi)
            x_proj = w / 2 + (focal_length * x_sphere * w / (2 * z_sphere))
            y_proj = h / 2 + (focal_length * y_sphere * h / (2 * z_sphere))
            map_x[y, x] = x_proj
            map_y[y, x] = y_proj

# 进行球形投影
spherical_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

# 显示结果
cv2.imwrite('Equirectangular_Projection.jpg', spherical_image)
cv2.imwrite('ori.jpg', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
